In [27]:
import geopandas as gpd
import numpy as np
import pandas as pd
import hyp3_sdk as sdk
import asf_search as asf
import pyproj
import json
import rasterio as rio
import matplotlib.pyplot as plt
import seaborn as sns
import sys

sys.path.append('../')

from PIL import Image
from shapely.geometry import Polygon
from config import hyp3_username, hyp3_password

%matplotlib inline


pd.options.display.max_columns = 2000
pd.set_option('display.max_colwidth',1000)

In [28]:
df = pd.read_excel('/Users/kaipak/Desktop/bit bucket/SLICE_bitbucket/WESTMED_MG_engl.xlsx')

In [29]:
df.head()

,File Name,Date and Time,Center Coordinate,Eastern Auxiliary Coordinate,Diameter long [km],Diameter short [km],Rotation,Type,Image?,Notes,Lage,Date,Unnamed: 12,Shape,Unnamed: 14,Eccentricity,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,S1A_EW_GRDH_1SDH_20141006T060901_20141006T061005_002705_003068_E455,2014-10-06 06:09:01,+0.1905°O 39.9804°N,+0.2088°O 39.9935°N,1.90,1.22,cyclonic,S,no,NaN,NaN,2014-10-01,NaN,0.642105,0.0,0.766616,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S1A_EW_GRDH_1SDH_20141006T060901_20141006T061005_002705_003068_E455,2014-10-06 06:09:01,+0.3771°O 39.8476°N,+0.4352°O 39.8365°N,4.35,2.34,cyclonic,S,no,NaN,NaN,2014-10-02,NaN,0.537931,NaN,0.842989,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S1A_EW_GRDH_1SDH_20141006T060901_20141006T061005_002705_003068_E455,2014-10-06 06:09:01,+0.4663°O 40.1961°N,+0.4831°O 40.2057°N,3.57,2.65,cyclonic,S,no,NaN,NaN,2014-10-03,NaN,0.742297,NaN,0.670071,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S1A_EW_GRDH_1SDH_20141006T060901_20141006T061005_002705_003068_E455,2014-10-06 06:09:01,+0.1464°O 39.7419°N,+0.1659°O 39.7567°N,5.24,5.18,cyclonic,S,no,NaN,NaN,2014-10-04,NaN,0.988550,NaN,0.150896,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.695137
4,S1A_EW_GRDH_1SDH_20141006T060901_20141006T061005_002705_003068_E455,2014-10-06 06:09:01,+0.3086°O 39.8039°N,+0.3432°O 39.7766°N,6.25,4.59,anticyclonic,S,no,NaN,NaN,2014-10-05,NaN,0.734400,NaN,0.678717,NaN,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
foo = df.groupby(by='File Name').size().to_frame('count').reset_index()

In [31]:
foo.loc[foo['count'] > 1]

,File Name,count
0,S1A_EW_GRDH_1SDH_20141006T060901_20141006T061005_002705_003068_E455,6
1,S1A_EW_GRDH_1SDH_20141006T061005_20141006T061105_002705_003068_A8A6,6
3,S1A_EW_GRDH_1SDH_20141016T062652_20141016T062746_002851_00338B_01E8,2
4,S1A_EW_GRDH_1SDV_20141017T180916_20141017T181011_002873_0033FA_7339,8
5,S1A_EW_GRDH_1SDV_20141018T060901_20141018T061005_002880_00342B_C2DD,13
...,...,...
2074,S1A_IW_GRDH_1SDV_20170628T060940_20170628T061005_017230_01CBEE_B64C,2
2076,S1A_IW_GRDH_1SSV_20141122T052914_20141122T052943_003390_003F2A_A3E4,4
2077,S1A_IW_GRDH_1SSV_20141221T053734_20141221T053803_003813_0048EE_7E14,2
2078,S1A_IW_GRDH_1SSV_20150114T053733_20150114T053802_004163_0050C9_1E71,3


In [23]:
foo

,File Name,count
0,S1A_EW_GRDH_1SDH_20141006T060901_20141006T0610...,6
1,S1A_EW_GRDH_1SDH_20141006T061005_20141006T0611...,6
2,S1A_EW_GRDH_1SDH_20141008T055259_20141008T0553...,1
3,S1A_EW_GRDH_1SDH_20141016T062652_20141016T0627...,2
4,S1A_EW_GRDH_1SDV_20141017T180916_20141017T1810...,8
...,...,...
2076,S1A_IW_GRDH_1SSV_20141122T052914_20141122T0529...,4
2077,S1A_IW_GRDH_1SSV_20141221T053734_20141221T0538...,2
2078,S1A_IW_GRDH_1SSV_20150114T053733_20150114T0538...,3
2079,S1A_S2_GRDH_1SDH_20141122T180928_20141122T1809...,1
